In [1]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"

currentpath = michaelpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON
import DataStructures: PriorityQueue, enqueue!, dequeue!

include(".//src//model.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
 Activating environment at `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021

reassign_ambulances! (generic function with 2 methods)

In [18]:
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 2 methods)

In [19]:
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [20]:
using AmbulanceDeployment, Dates

In [21]:
Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue,guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)

        end
    end

end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        ems.eventlog[id, :responsetime] = travel_time / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*2*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
end


  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`


done_event! (generic function with 1 method)

In [22]:
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
calling event id: 2 time: 575 value: 38
Ambulance has arrived on the scene for event 1 at time 638
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 6 at time 1787
Ambulance for event 1 has arrived at the hospital at time 1836
calling event id: 10 time: 1838 value: 67
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance has arrived on the scene for event 4 at time 1912
Ambulance has arrived on the scene for event 2 at time 1924
calling event id: 13 time: 2091 value: 98
calling event id: 14 time: 2117 value: 113
Ambulance has arrived on the scene for event 3 at time 2140
Ambulance has arrived on the scene for event 8 at time 2178


Ambulance has arrived on the scene for event 43 at time 12660
calling event id: 45 time: 12678 value: 99
Ambulance has arrived on the scene for event 44 at time 12679
Ambulance for event 40 has arrived at the hospital at time 12778
Ambulance for event 39 has arrived at the hospital at time 12824
calling event id: 46 time: 12885 value: 124
Ambulance for event 37 has arrived at the hospital at time 12994
Ambulance for event 38 has arrived at the hospital at time 12995
calling event id: 47 time: 13075 value: 144
Ambulance for event 35 is returning to the station at 13085
Ambulance for event 36 is returning to the station at 13115
Ambulance has arrived on the scene for event 45 at time 13278
Ambulance for event 42 has arrived at the hospital at time 13291
Ambulance for event 43 has arrived at the hospital at time 13436
Ambulance for event 41 has arrived at the hospital at time 13478
Ambulance has arrived on the scene for event 46 at time 13485
Ambulance from event 35 has returned to statio

Ambulance from event 88 has returned to station 9
Ambulance for event 85 is returning to the station at 31871
Ambulance for event 87 is returning to the station at 31987
Ambulance for event 90 has arrived at the hospital at time 31988
Ambulance for event 89 is returning to the station at 32252
calling event id: 93 time: 32356 value: 66
Ambulance from event 84 has returned to station 17
Ambulance for event 91 is returning to the station at 32558
Ambulance for event 92 has arrived at the hospital at time 32595
Ambulance from event 81 has returned to station 30
Ambulance from event 91 has returned to station 8
Ambulance has arrived on the scene for event 93 at time 32956
calling event id: 94 time: 32957 value: 66
Ambulance from event 89 has returned to station 25
Ambulance from event 83 has returned to station 44
Ambulance has arrived on the scene for event 94 at time 33557
Ambulance for event 92 is returning to the station at 33795
Ambulance for event 93 has arrived at the hospital at ti

Ambulance for event 130 has arrived at the hospital at time 47816
calling event id: 133 time: 47904 value: 131
133: call from 131 is a shortfall
Ambulance for event 123 is returning to the station at 48000
Ambulance from event 118 has returned to station 8
calling event id: 134 time: 48031 value: 66
Ambulance for event 117 is returning to the station at 48142
Ambulance for event 129 has arrived at the hospital at time 48155
Ambulance for event 121 is returning to the station at 48244
Ambulance for event 124 has arrived at the hospital at time 48311
Ambulance from event 122 has returned to station 3
Ambulance for event 125 has arrived at the hospital at time 48452
Ambulance from event 120 has returned to station 11
Ambulance has arrived on the scene for event 131 at time 48479
Ambulance from event 123 has returned to station 34
Ambulance for event 127 is returning to the station at 48624
Ambulance has arrived on the scene for event 134 at time 48631
Ambulance for event 128 has arrived a

Ambulance for event 165 is returning to the station at 56573
Ambulance for event 169 has arrived at the hospital at time 56647
Ambulance for event 162 has arrived at the hospital at time 56657
Ambulance from event 157 has returned to station 18
Ambulance for event 156 is returning to the station at 56687
Ambulance for event 159 has arrived at the hospital at time 56783
Ambulance from event 155 has returned to station 25
Ambulance from event 150 has returned to station 28
Ambulance from event 152 has returned to station 22
calling event id: 171 time: 56879 value: 36
Ambulance for event 154 is returning to the station at 56973
Ambulance for event 168 has arrived at the hospital at time 57061
calling event id: 172 time: 57150 value: 98
Ambulance for event 160 is returning to the station at 57253
Ambulance from event 158 has returned to station 17
calling event id: 173 time: 57294 value: 98
calling event id: 174 time: 57330 value: 110
calling event id: 175 time: 57347 value: 113
Ambulance 

...

Ambulance for event 846 has arrived at the hospital at time 244596
Ambulance for event 850 has arrived at the hospital at time 244691
Ambulance for event 840 is returning to the station at 244706
calling event id: 863 time: 244736 value: 37
Ambulance for event 843 is returning to the station at 244882
Ambulance from event 841 has returned to station 17
Ambulance has arrived on the scene for event 854 at time 244958
Ambulance for event 855 has arrived at the hospital at time 244989
calling event id: 864 time: 245011 value: 113
Ambulance has arrived on the scene for event 862 at time 245016
Ambulance for event 857 has arrived at the hospital at time 245047
Ambulance has arrived on the scene for event 852 at time 245063
calling event id: 865 time: 245074 value: 128
calling event id: 866 time: 245099 value: 68
calling event id: 867 time: 245141 value: 67
Ambulance has arrived on the scene for event 859 at time 245175
Ambulance for event 842 is returning to the station at 245219
Ambulance f

Ambulance for event 883 is returning to the station at 252414
calling event id: 896 time: 252607 value: 52
calling event id: 897 time: 252620 value: 51
Ambulance has arrived on the scene for event 891 at time 252636
calling event id: 898 time: 252637 value: 129
Ambulance from event 883 has returned to station 8
calling event id: 899 time: 252799 value: 68
calling event id: 900 time: 252837 value: 71
calling event id: 901 time: 252877 value: 129
Ambulance for event 885 is returning to the station at 252877
Ambulance from event 872 has returned to station 2
Ambulance for event 886 is returning to the station at 253041
Ambulance for event 888 is returning to the station at 253071
calling event id: 902 time: 253137 value: 67
Ambulance for event 884 is returning to the station at 253152
calling event id: 903 time: 253176 value: 83
903: call from 83 is a shortfall
calling event id: 904 time: 253220 value: 82
Ambulance from event 885 has returned to station 17
Ambulance from event 882 has ret

calling event id: 936 time: 261026 value: 68
Ambulance has arrived on the scene for event 931 at time 261031
Ambulance for event 926 has arrived at the hospital at time 261097
Ambulance has arrived on the scene for event 932 at time 261111
Ambulance from event 913 has returned to station 17
calling event id: 937 time: 261125 value: 129
937: call from 129 is a shortfall
Ambulance from event 916 has returned to station 5
Ambulance from event 919 has returned to station 8
Ambulance for event 930 has arrived at the hospital at time 261285
calling event id: 938 time: 261315 value: 68
Ambulance from event 917 has returned to station 19
Ambulance from event 910 has returned to station 2
Ambulance has arrived on the scene for event 933 at time 261413
Ambulance from event 895 has returned to station 3
Ambulance for event 928 has arrived at the hospital at time 261479
Ambulance for event 920 is returning to the station at 261546
Ambulance for event 923 has arrived at the hospital at time 261581


Ambulance from event 959 has returned to station 5
Ambulance has arrived on the scene for event 969 at time 269049
calling event id: 970 time: 269088 value: 52
Ambulance for event 965 has arrived at the hospital at time 269116
Ambulance for event 957 is returning to the station at 269126
Ambulance for event 964 has arrived at the hospital at time 269128
calling event id: 971 time: 269150 value: 69
calling event id: 972 time: 269164 value: 156
Ambulance for event 960 is returning to the station at 269287
Ambulance for event 963 is returning to the station at 269432
Ambulance from event 955 has returned to station 4
Ambulance for event 962 is returning to the station at 269463
Ambulance for event 969 has arrived at the hospital at time 269576
Ambulance from event 960 has returned to station 8
calling event id: 973 time: 269600 value: 127
Ambulance for event 968 has arrived at the hospital at time 269934
Ambulance for event 961 is returning to the station at 270025
Ambulance from event 95

LoadError: [91mMethodError: no method matching iterate(::Nothing)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  iterate([91m::MathOptInterface.Bridges.Objective.Map[39m, [91m::Any...[39m) at /home/mhilborn/.julia/packages/MathOptInterface/ZJFKw/src/Bridges/Objective/map.jl:30[39m
[91m[0m  iterate([91m::MutableArithmetics.Zero[39m) at /home/mhilborn/.julia/packages/MutableArithmetics/bPWR4/src/rewrite.jl:88[39m
[91m[0m  iterate([91m::MutableArithmetics.Zero[39m, [91m::Nothing[39m) at /home/mhilborn/.julia/packages/MutableArithmetics/bPWR4/src/rewrite.jl:89[39m
[91m[0m  ...[39m

1000-element Array{Float64,1}:
 12.997833333333334
 15.806166666666666
 24.017333333333333
 36.8475
 17.139166666666664
 14.4925
 36.14666666666667
  5.0
 11.344
  5.0
  5.0
 12.158
 15.441166666666668
  ⋮
 22.715666666666667
 19.34
 12.997833333333334
 22.715666666666667
 22.715666666666667
 17.139166666666664
 15.652
 17.139166666666664
 20.8855
 24.1
 15.652
 12.158